In [ ]:
DESCRIPTION = """Using quantum espresso to compute the stresses associated with different strain states and fitting the elastic tensor."""

In [ ]:
# Import libraries
import numpy as np
from mp_api.client import MPRester
from pymatgen.io.pwscf import PWInput
from pymatgen.core import Structure
from pymatgen.analysis.elasticity import *
from pymatgen.io.vasp.inputs import *
from pymatgen.core.tensors import symmetry_reduce
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os

from simtool import getValidatedInputs, DB

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

mp_id:
    type: Text
    description: Materials Project ID for chosen structure
        
structure_dict:
    type: Dict
    description: Pymatgen structure object saved as dictionary

num_atoms:
    type: Integer
    description: Total number of atoms in the simulation cell
    min: 0
    max: 200
        
n_atom_types:
    type: Integer
    description: Number of elements in compostion. Examples Si - 1, TiN - 2, BaZrO3 - 3
        
space_group:
    type: Integer
    description: Space group number, i.e. 225
    value: 0
    min: 0
    max: 230
        
is_metal:
    type: Boolean
    description: Is the system considered assumed to be metallic or not. Used to include occupation information for the QE calculations.
    value: False
        
KE_cutoff:
    type: Number
    description: Kinetic energy cutoff (Ry)
    value: 50
    min: 10
    max: 300
    units: rydberg
        
k_x:
    type: Integer
    description: Number of k-points in x direction
    value: 1
    min: 1
    max: 30

k_y:
    type: Integer
    description: Number of k-points in y direction
    value: 1
    min: 1
    max: 30
    
k_z:
    type: Integer
    description: Number of k-points in z direction
    value: 1
    min: 1
    max: 30
        
i_steps:
    type: Integer
    description: Maximum number of ionic steps during relaxation
    value: 50
    
e_steps:
    type: Integer
    description: Maximum number of electronic steps during SCF calculation
    value: 100
    min: 100
    max: 1000
        
scf_conv:
    type: Number
    description: Convergence threshold for selfconsistency
    value: 1e-10
        
energy_conv:
    type: Number
    description: Convergence threshold on total energy for ionic minimization
    value: 1e-10
    min: 1e-15
    max: 1e-4
        
force_conv:
    type: Number
    description: Convergence threshold on forces for ionic minimization 
    value: 1e-5
    min: 1e-8
    max: 1e-3
        
vdw_corr:
    type: Choice
    description: Type of vdw correction. Options are 'grimme-d2', 'grimme-d3', or 'none'.
    options: ['grimme-d2','grimme-d3','none']
    value: 'none'
        
spin_polar:
    type: Boolean
    description: This parameter controls whether a spin-polarized calculation is run. False for non-polarized and True for spin-polarized, LSDA (magnetization along z axis)
    value: False
        
hubbard_U_on:
    type: Boolean
    description: This parameter controls whether a the Hubbard U correction is added to the calculation to account for localized d or f electrons that are not well described by standard DFT. False for no Hubbard U correction and True for adding the Hubbard U correction
    value: False
        
hubbard_projector:
    type: Choice
    description: This will determine the type of Hubbard projector used in the calculation. Options are 'atomic', 'ortho-atomic', 'norm-atomic', 'wf', or 'pseudo'. If the hubbard_U_on input is set to False this input is ignored when creating the QE input file. It is highly recommended to use ortho-atomic or atomic whenever possbile.
    options: ['atomic','ortho-atomic','norm-atomic','wf','pseudo']
    value: 'ortho-atomic'
        
hubbard_U_values:
    type: Dict
    description: This is a dictionary that will have the value for the U parameter in eV for each atomic species. 
    
hubbard_U_orbitals:
    type: Dict
    description: This is a dictionary that will have the orbitals that the U parameter is acting on for each atomic species.

pseudo:
    type: Choice
    description: Controls the type of pseudopotential used for the calculation
    options: ['PAW','USPP']
    value: 'PAW'
        
strain_matrices:
    type: Array
    description: Array containing the strain matrices as arrays that will be used to generate the deformed structures and fit the elastic tensor
        
sym_red:
    type: Boolean
    description: Decides whether to reduce the number of deformations based on symmetry
    value: False
        
external_pressure:
    type: Number
    description: The value for the external pressure of the pristine structure
    value: 0.0
    units: GPa

In [ ]:
defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)

In [ ]:
EXTRA_FILES = ['pseudo']

In [ ]:
%%yaml OUTPUTS

strain_matrices:
    type: Array
    description: Array of strain matrices that were actually used in the generation of the deformed structures and fitting of the elastic tensor (only different from the input if sym_red is True)

deformed_structures:
    type: List
    description: List of pymatgen Structure objects in dictionary format that were generated by the applying strains to the pristine structure
        
stress_tensors:
    type: Array
    description: Array of stress tensors as arrays that were computed by quantum espresso for each deformed structure and used in the fit for the elastic tensor
    
elastic_tensor:
    type: Array
    description: Elastic tensor in voigt notation fit with pymatgen using the Moore-Penrose pseudo-inverse method
        
property_dict:
    type: Dict
    description: Dictionary of various properties derived from the elastic tensor

In [ ]:
struct = Structure.from_dict(structure_dict,'cif')

In [ ]:
def make_sim(name,struct,sym,**kwargs):
    """
    Generate quantum espresso input files using pymatgen's PWInput class
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    **kwargs:
        dictionaries to input to pymatgen's PWInput object
    """
    # Prepare dict of pseudopotentials (i.e. {'Mg': 'Mg.upf', 'O': 'O.upf'})
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_dict = dict(zip(elements,[f"{element}.upf" for element in elements]))

    # Define input set
    input_set = PWInput(structure=struct,
                        pseudo=pseudo_dict,
                        **kwargs) # dictionaries corresponding to blocks in QE input files

    input_set.write_file(filename=f'{name}.in')
    # if we want to impose symmetry we remove the CELL_PARAMETERS card from qe in file
    if sym != 0:      
        with open(f'{name}.in') as f1:
            lines = f1.readlines()

        with open('tmp.in', 'w') as f2:
            f2.writelines(lines[:-4])
            
        os.rename('tmp.in', f'{name}.in')
    
def run_sim(name,struct,pseudo):
    """
    Submit quantum espresso runs to HPC clusters on nanoHUB
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    """
    # Write input and output files
    input_file = open(f'{name}.in','a')
    input_file.close()

    output_file = open(f'{name}.out', 'w')
    output_file.close()
    
    # Set up commands and files
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_arg = "".join([f"-i ./pseudo/pseudo_{pseudo}_PBEsol/{element}.upf " for element in elements])
    COMMAND = f"espresso-7.1_pw > {output_file.name}"
    
    # Run simulation (1 node, 64 cpus, 24 hour walltime)
    !submit -n 64 -w '24:00:00' --noquota -e QE_DISABLE_GGA_PBE=0 --runName {name} {pseudo_arg} {COMMAND} -i {input_file.name}  

# Define helper functions for QE    
def get_qe_outputs_relax(file):
    """
    Extract outputs (energies, forces, structures) from qe .stdout files
    
    inputs:
        file: path to the file we want to extract outputs from
    outputs:
        dict: dictionary of the extracted outputs
    """
    
    output = open(file, "r")
    lines = output.readlines()
    iE = [] # energy at each ionic step, Ry
    eE = [[]] # energy at each electronic step, Ry
    P = [] # pressure, kbar
    F = [] # total force, Ry/au
    stresses = [] # stress tensor, kbar
    structures = [] # pymatgen structure objects, angstrom

    # Check for certain tags on lines, add variables to lists
    for i,line in enumerate(lines):
        if 'total energy' in line and '!' not in line and 'The' not in line:
            eE[-1].append(float(line.split()[3]))
        elif 'total energy' in line and '!' in line:
            eE.append([])
            iE.append(float(line.split()[4]))
        elif 'P=' in line:
            P.append(float(line.split()[5]))
            stresses.append(np.array([lines[i+1].split()[3:6],lines[i+2].split()[3:6],lines[i+3].split()[3:6]]).astype(float))
        elif "Total force" in line:
            F.append(float(line.split()[3]))
        elif 'CELL_PARAMETERS' in line:
            try:
                if 'alat' in line:
                    scale = float(line.split()[-1].split(')')[0])*0.529177
                else:
                    scale = 1.0
                lattice = scale*np.array([lines[i+1].split(),lines[i+2].split(),lines[i+3].split()]).astype(float)
                sites = []
                atoms = []
                j=6
                while ("End" not in lines[i+j].strip()) and (lines[i+j].strip()!=""):
                    sites.append(np.array(lines[i+j].split()[1:]).astype(float))
                    atoms.append(lines[i+j].split()[0])
                    j=j+1
                lattice_obj = Lattice(lattice)
                print(lattice, atoms, sites)
                test_struct = Structure(lattice,atoms,sites)
                structures.append(test_struct)
            except:
                pass
    eE = eE[:-1]

    # return output dictionary
    return {'ionic_energies':iE,'electronic_energies':eE,'pressures':P,'forces':F,'stresses':stresses,'structures':structures}

In [ ]:
electron_dict = {'electron_maxstep':e_steps,'conv_thr':scf_conv}

if is_metal:
    system_dict = {'ecutwfc':int(KE_cutoff),'occupations':'smearing','smearing':'gauss','degauss':0.02,'vdw_corr':vdw_corr}
else:
    system_dict = {'ecutwfc':int(KE_cutoff),'vdw_corr':vdw_corr}
    
if spin_polar:
    electron_dict['mixing_beta']=0.3
    system_dict['nspin']=2
    for i in range(n_atom_types):
        system_dict['starting_magnetization('+str(i+1)+')']=0.6
        if not is_metal:
            system_dict['occupations']='smearing'
            system_dict['smearing']='gauss'
            system_dict['degauss']=0.02
            
def add_hubbard(name,struct,hubbard_projector,hubbard_U_values):
    h_lines = ['HUBBARD {'+hubbard_projector+'} \n']
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    for e in elements:
        u = hubbard_U_values[e]
        o = hubbard_U_orbitals[e] 
        h_lines.append(f'U {e}-{o} {u}\n')
            
    f = open(f'{name}.in','r')
    lines = f.readlines()
    for line in h_lines:
        lines.append(line)
    f_new = open('tmp.in','w')
    f_new.writelines(lines)
    f.close()
    f_new.close()
    
    os.rename('tmp.in', f'{name}.in')
    
if hubbard_U_on:
    electron_dict['mixing_beta']=0.3

In [ ]:
# Create pymatgen Strain objects from matrices
strain_mats = [Strain(s) for s in strain_matrices]

# Generate deformed structures
deformations = []

for s_mat in strain_mats:
    deform_matrix = s_mat.get_deformation_matrix()
    deformations.append(deform_matrix)

if sym_red:
    sym_dict = symmetry_reduce(deformations, struct)
    deformations = list(sym_dict)

    strain_mats_reduced = [Strain.from_deformation(defo) for defo in deformations]
    strain_mats = strain_mats_reduced
    
deform_structs = [defo.apply_to_structure(struct) for defo in deformations]
deform_struct_dicts = [defo_struct.as_dict() for defo_struct in deform_structs]
    
# Calculate each stress tensor
system_dict = {'ecutwfc':KE_cutoff,'vdw_corr':vdw_corr}
electron_dict = {'electron_maxstep':e_steps,'conv_thr':scf_conv}

stress_tensors = []
stress_tensor_arrays = []
for i,d_struct in enumerate(deform_structs):
    print(f'Running calculation on deformed structure {i} out of {len(deform_structs)}')

    make_sim("relax", d_struct, sym=0,
             control={'pseudo_dir':'./',
                      'calculation':'relax',
                      'outdir':'./',
                      'tstress':True,
                      'nstep':i_steps,
                      'etot_conv_thr':energy_conv,
                      'forc_conv_thr':force_conv,
                      'disk_io':'nowf'},
             system=system_dict,
             electrons=electron_dict,
             kpoints_grid=[k_x,k_y,k_z])
    
    # Add Hubbard U correction if needed
    if hubbard_U_on:
        add_hubbard("relax", struct, hubbard_projector, hubbard_U_values)

    run_sim("relax",d_struct,pseudo)
    
    try:
        relax_dict = get_qe_outputs_relax('relax.stdout')
    except:
        print('Getting Relax Dictionary Failed')
        
    s_tensor = -Stress(relax_dict['stresses'][-1]/10)  # convert kbar to eV/angs^3
    stress_tensors.append(s_tensor)
    stress_tensor_arrays.append(np.array(s_tensor))
    
    os.mkdir(f'strain_{i}-{len(deform_structs)}')
    os.system(f'mv relax* strain_{i}-{len(deform_structs)}')
    os.system('rm -r pw*')

In [ ]:
# Fit elastic tensor
elastic_tensor = ElasticTensor.from_pseudoinverse(strains=strain_mats,stresses=stress_tensors)

In [ ]:
property_dict = elastic_tensor.get_structure_property_dict(struct)

In [ ]:
db = DB(OUTPUTS)

In [ ]:
db.save('strain_matrices',np.array(strain_mats))
db.save('deformed_structures', deform_struct_dicts)
db.save('stress_tensors', np.array(stress_tensor_arrays))
db.save('elastic_tensor', elastic_tensor.voigt)
db.save('property_dict', property_dict)